In [2]:
import numpy as np
from scipy.constants import physical_constants as pc

## Puits quantique asymétrique


In [31]:
def get_H_mn(m, n, a=10, L=5e-10):
    hbar = pc['Planck constant over 2 pi'][0]
    m_e = pc['electron mass'][0]
    
    # Résultat du terme de qte de mvt
    tmp = (n*np.pi*hbar)**2/(4*m_e*L)
    
    
    # Résultat du terme de potentiel
    if m != n:

        # Si les deux sont simultanément pairs ou impairs
        if ((m%2 == 0) and (n%2 == 0)) or ((m%2 != 0) and (n%2 != 0)):

            H_mn = 0

        # Si l'un est pair et l'autre est impair, et inversement
        elif ((m%2 == 0) and (n%2 != 0)) or ((m%2 != 0) and (n%2 == 0)):
            H_mn = -(a*4*L/np.pi**2)*(m*n/(m**2 - n**2))
    
    # Si m = n
    else: 
        H_mn = a*L/4 + tmp
            
    return 2*(H_mn)/L




def get_H(m, n):
    
    H = np.zeros((m, n))
    
    for i in range(m):
        for j in range(n):
            
            H[i, j] = get_H_mn(i+1, j+1)
            
    return H


np.set_printoptions(threshold=np.nan)

H_mn = get_H_mn(2, 1)
print(H_mn)
H = get_H(10, 10)
print(H)

            
                

-5.4037964609246805
[[  5.           5.40379646   0.           2.16151858   0.           1.38954766
    0.           1.02929456   0.           0.81875704]
 [ -5.40379646   5.           9.72683363   0.           3.85985461   0.
    2.52177168   0.           1.89483772   0.        ]
 [  0.          -9.72683363   5.          13.89547661   0.           5.40379646
    0.           3.53703041   0.           2.67220704]
 [ -2.16151858   0.         -13.89547661   5.          18.01265487   0.
    6.87755913   0.           4.48930783   0.        ]
 [  0.          -3.85985461   0.         -18.01265487   5.          22.10644007
    0.           8.31353302   0.           5.40379646]
 [ -1.38954766   0.          -5.40379646   0.         -22.10644007   5.
   26.187629     0.           9.72683363   0.        ]
 [  0.          -2.52177168   0.          -6.87755913   0.         -26.187629
    5.          30.26126018   0.          11.1254633 ]
 [ -1.02929456   0.          -3.53703041   0.          -8.313